## Kitchen training - test

In [1]:
import gdown
from sklearn.model_selection import train_test_split
import os
import shutil
import re
import pandas as pd

OD_PATH = '/models/research/object_detection/'
MAIN_PATH = '/happy-walrus/models/version_3/'
IMG_PATH = MAIN_PATH + 'images/'
MODEL_NAME = 'autoaugm_resnet101/'
TRAIN_PATH = MAIN_PATH + 'training/' + MODEL_NAME
EVAL_PATH = MAIN_PATH + 'evaluation/' + MODEL_NAME
GRAPH_PATH = MAIN_PATH + 'inference_graph/' + MODEL_NAME

### Resources

- [Tensorflow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection)
- [Tensorflow Object Detection API tutorial](https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10)

# 1. Training

### Download data

Store all images and annotations in a single folder, zip the folder and store it in Google Drive under `safety/data/images/version_X` (replace the `X`).

In [2]:
# Download images
file_id = "16PHxTQs2JW8Zq1HpsscBLcQuWcFGG_gR"
file_name = "version_3.zip"
gdown.download(url="https://drive.google.com/uc?id=" + file_id,
               output=MAIN_PATH + file_name,
               quiet=False)

# Unzip images
os.mkdir(IMG_PATH)
!unzip -q {MAIN_PATH}{file_name} -d {IMG_PATH}
!rm {MAIN_PATH}version_3.zip
!cd {MAIN_PATH}images && ls | grep "(1)" | xargs -d"\n" rm
files_unzipped = os.listdir(MAIN_PATH + 'images/')

# Print results
print('Files unzipped:', len(files_unzipped))
print('XML files:', len([file for file in files_unzipped if re.match('.*\.xml', file)]))
print('JPG files:', len([file for file in files_unzipped if re.match('.*\.jpg', file)]))

Downloading...
From: https://drive.google.com/uc?id=16PHxTQs2JW8Zq1HpsscBLcQuWcFGG_gR
To: /happy-walrus/models/version_3/version_3.zip
580MB [00:06, 90.1MB/s] 


Files unzipped: 5757
XML files: 2881
JPG files: 2876


Randomly split data into train and test:
```
|-images
    |- train
    |- test
```
where the last level folders will contain both the images and annotations (.xml) files.

In [3]:
# Create train and test folders in images/
os.mkdir(IMG_PATH + 'train')
os.mkdir(IMG_PATH + 'test')

# Get list of all files that have annotations and split into train and test
jpg_files = [file[:-4] for file in os.listdir(IMG_PATH) if re.search('.*\.jpg$', file)]
files = [file[:-4] for file in os.listdir(IMG_PATH) if re.search('.*\.xml$', file) and file[:-4] in jpg_files]
files = [file for file in files if not re.search('open_oven', file)]
files_train, files_test = train_test_split(files, train_size=0.9, test_size=0.1, random_state=2014)

# Move images to train or test folders
errors = []
for file in files_train:
    try:
        shutil.move(IMG_PATH + file + '.xml', IMG_PATH + 'train/' + file + '.xml')
        shutil.move(IMG_PATH + file + '.jpg', IMG_PATH + 'train/' + file + '.jpg')
    except Exception as err:
        errors.append([file, err])

for file in files_test:
    try:
        shutil.move(IMG_PATH + file + '.xml', IMG_PATH + 'test/' + file + '.xml')
        shutil.move(IMG_PATH + file + '.jpg', IMG_PATH + 'test/' + file + '.jpg')
    except Exception as err:
        errors.append([file, err])

# Remove remaining images
files_rm = [file for file in os.listdir(IMG_PATH) if re.search('.*\.jpg|.*\.xml', file)]
for file in files_rm:
    os.remove(IMG_PATH + file)
        
# Print errors
print("Removed files:", len(files_rm))
print("Number of errors:", len(errors))
print(errors)

Removed files: 69
Number of errors: 0
[]


### Generate data

Generate training and test data to `./images/`

In [4]:
# Generate training and test data
!python {OD_PATH}xml_to_csv.py
print("")
!head {IMG_PATH}train_labels.csv -n 5

# Filter ladders out, fix utensil(s)
print("")
train_csv = pd.read_csv(IMG_PATH + 'train_labels.csv')
train_csv = train_csv[train_csv['class'] != 'ladder']
train_csv['class'] = train_csv['class'].map(lambda x: 'utensil' if x == 'utensils' else x)
train_csv.to_csv(IMG_PATH + 'train_labels.csv')

test_csv = pd.read_csv(IMG_PATH + 'test_labels.csv')
test_csv = test_csv[test_csv['class'] != 'ladder']
test_csv['class'] = test_csv['class'].map(lambda x: 'utensil' if x == 'utensils' else x)
test_csv.to_csv(IMG_PATH + 'test_labels.csv')
unique_classes = sorted(train_csv['class'].unique())

# Print results
print("Unique classes:", unique_classes)
print("")
print("Train images:", train_csv['filename'].unique().shape[0])
print(f"Train annotations: {train_csv.shape[0]:,}")
print("Annotations per class (training):")
print(train_csv['class'].value_counts())
print("")
print("Test images:", test_csv['filename'].unique().shape[0])
print(f"Test annotations: {test_csv.shape[0]:,}")
print("")

# Generate label-id mapping
class_mapping = []
for count, item in enumerate(unique_classes):
    class_mapping.append('item{')
    class_mapping.append('\tid: ' + str(count + 1))
    class_mapping.append('\tname: \'' + item + '\'')
    class_mapping.append('}')
with open(MAIN_PATH + 'training/labelmap.pbtxt', 'w') as f:
    f.write('\n'.join(class_mapping))
print("Label id mapping:")
print('\n'.join(class_mapping))

Successfully converted xml to csv.
Successfully converted xml to csv.

filename,width,height,class,xmin,ymin,xmax,ymax
kitchen_apartment.NM.048.jpg,1280,720,stool,786,621,959,678
kitchen_apartment.NM.048.jpg,1280,720,oven,749,466,886,580
kitchen_apartment.NM.048.jpg,1280,720,countertop,356,422,893,460
purple_kitchen.EH.025.jpg,1000,717,stove,247,358,384,368

Unique classes: ['cabinet', 'chair', 'countertop', 'dishwasher', 'outlet', 'oven', 'sofa', 'stool', 'stove', 'utensil']

Train images: 2539
Train annotations: 18,949
Annotations per class (training):
countertop    4622
cabinet       3543
chair         2669
outlet        2327
stove         1995
utensil       1459
oven           960
stool          692
dishwasher     584
sofa            98
Name: class, dtype: int64

Test images: 284
Test annotations: 2,058

Label id mapping:
item{
	id: 1
	name: 'cabinet'
}
item{
	id: 2
	name: 'chair'
}
item{
	id: 3
	name: 'countertop'
}
item{
	id: 4
	name: 'dishwasher'
}
item{
	id: 5
	name: 'outlet'
}

In [7]:
!head generate_tfrecord.py -n 53 | tail -n 23

def class_text_to_int(row_label):
    if row_label == 'cabinet':
        return 1
    elif row_label == 'chair':
        return 2
    elif row_label == 'countertop':
        return 3
    elif row_label == 'dishwasher':
        return 4
    elif row_label == 'outlet':
        return 5
    elif row_label == 'oven':
        return 6
    elif row_label == 'sofa':
        return 7
    elif row_label == 'stool':
        return 8
    elif row_label == 'stove':
        return 9
    elif row_label == 'utensil':
        return 10 
    else:
        None


Copy `generate_tfrecord.py` to main folder and:
1. replace `import tensorflow` with `import tensorflow.compat.v1`.
2. Modify `class_test_to_int()` function with label-id mapping.
3. Line 91: add `'image/object/difficult': dataset_util.int64_list_feature([0]*len(xmins)),`

In [5]:
# Generate TF formatted data
!python generate_tfrecord.py --csv_input={IMG_PATH}train_labels.csv --image_dir={IMG_PATH}train --output_path=train.record
!python generate_tfrecord.py --csv_input={IMG_PATH}test_labels.csv --image_dir={IMG_PATH}test --output_path=test.record
print("")
!ls -lh | grep .*\.record$

/root/.local/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))
Successfully created the TFRecords: /happy-walrus/models/version_3/train.record
Successfully created the TFRecords: /happy-walrus/models/version_3/test.record

-rw-r--r-- 1 root root  60M Nov 27 16:13 test.record
-rw-r--r-- 1 root root 489M Nov 27 16:13 train.record


### Training

Get model checkpoint (other models in Tensorflow object detection's [Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)). Models:
- [faster rcnn inception v2 coco](http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz)
    - Speed: 58
    - mAP: 28
- [faster rcnn resnet101 coco](http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz)
    - Speed: 106
    - mAP: 32
- [inception resnet v2 atrous](http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28.tar.gz)
    - Speed: 620
    - mAP: 37
- [faster_rcnn_inception_resnet_v2_atrous_oid_v4](http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12.tar.gz)
    - Speed: 425
    - mAP: 54

In [6]:
!wget "http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz"
!tar xvzf faster_rcnn_resnet101_coco_2018_01_28.tar.gz

--2019-11-27 16:14:39--  http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.7.144, 2607:f8b0:4004:814::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.7.144|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 593445323 (566M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet101_coco_2018_01_28.tar.gz’

faster_rcnn_resnet1 100%[===================>] 565.95M  31.7MB/s    in 19s     

2019-11-27 16:15:05 (29.5 MB/s) - ‘faster_rcnn_resnet101_coco_2018_01_28.tar.gz’ saved [593445323/593445323]

faster_rcnn_resnet101_coco_2018_01_28/
faster_rcnn_resnet101_coco_2018_01_28/model.ckpt.index
faster_rcnn_resnet101_coco_2018_01_28/checkpoint
faster_rcnn_resnet101_coco_2018_01_28/pipeline.config
faster_rcnn_resnet101_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_resnet101_coco_2018_01_28/model.ckpt.meta
faster_rcnn_re

1. Copy config file from `./samples/configs/` to `./training/`
2. Adapt parameters in config file:
    - `num_classes`
    - model `type`
    - `fine_tune_checkpoint`
    - train_input_reader: `input_path`, `label_map_path`
    - `num_examples`: images/test/
    - eval_input_reader: `input_path`, `label_map_path`
    - `data_augmentation_options`, see augmentations and arguments in [preprocessor module](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto).

In [72]:
# WARNING: this overwrites existing config file (copy only if no config already created)
!cp {OD_PATH}samples/configs/faster_rcnn_inception_v2_pets.config {MAIN_PATH}training/faster_rcnn_inception_v2_kitchens.config

Move train.py from `legacy/` to `.`

In [12]:
!cp {OD_PATH}legacy/train.py .

Train model
- Make sure to delete all files in `training/<model_name>` except the config file.
- Stops at 200.000 steps. Stop manually if necessary.

In [ ]:
CONFIG_PATH = TRAIN_PATH + MODEL_NAME[:-1] + '.config'
!python train.py \
    --logtostderr \
    --train_dir={TRAIN_PATH} \
    --pipeline_config_path={CONFIG_PATH}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.






Instructions for updating:
Use object_detection/model_main.py.
W1127 16:19:32.505128 140454575892288 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/absl/app.py:250: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.

W1127 16:19:32.505361 140454575892288 module_wrapper.py:139] From train.py:91: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1127 16:19:32.505721 140454575892288 module_wrapper.py:139] From /models/research/object_detection/utils/config_util.py:102: The name tf.gfile.G

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1127 16:19:33.386990 140454575892288 deprecation.py:323] From /models/research/object_detection/utils/autoaugment_utils.py:1212: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.
W1127 16:19:33.434649 140454575892288 deprecation.py:323] From /models/research/object_detection/utils/autoaugment_utils.py:898: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.
W1127 16:19:33.436315 140454575892288 deprecation.py:323] From /models/research/object_detection/utils/autoaugment_utils.py:900: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be


W1127 16:19:39.636150 140454575892288 module_wrapper.py:139] From /models/research/object_detection/core/minibatch_sampler.py:85: The name tf.random_shuffle is deprecated. Please use tf.random.shuffle instead.

Instructions for updating:
box_ind is deprecated, use box_indices instead
W1127 16:19:39.881497 140454575892288 deprecation.py:506] From /models/research/object_detection/utils/spatial_transform_ops.py:419: calling crop_and_resize_v1 (from tensorflow.python.ops.image_ops_impl) with box_ind is deprecated and will be removed in a future version.
Instructions for updating:
box_ind is deprecated, use box_indices instead

W1127 16:19:39.898820 140454575892288 module_wrapper.py:139] From /models/research/object_detection/meta_architectures/faster_rcnn_meta_arch.py:191: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

INFO:tensorflow:Scale of 0 disables regularizer.
I1127 16:19:39.899243 140454575892288 regularizers.py:98] Scale of 0 disables regulari


W1127 16:19:49.468914 140454575892288 module_wrapper.py:139] From /models/research/object_detection/utils/variables_helper.py:179: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W1127 16:19:49.469223 140454575892288 deprecation.py:323] From /models/research/object_detection/meta_architectures/faster_rcnn_meta_arch.py:2768: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step

W1127 16:19:49.472153 140454575892288 module_wrapper.py:139] From /models/research/object_detection/utils/variables_helper.py:139: The name tf.train.NewCheckpointReader is deprecated. Please use tf.compat.v1.train.NewCheckpointReader instead.

W1127 16:19:49.474493 140454575892288 variables_helper.py:157] Variable [Conv/biases/

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
W1127 16:19:50.570854 140454575892288 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/slim/python/slim/learning.py:742: Supervisor.__init__ (from tensorflow.python.training.supervisor) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
2019-11-27 16:19:52.218913: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-11-27 16:19:52.241218: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2600005000 Hz
2019-11-27 16:19:52.241859: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1179d0f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-11-27 16:19:52.241891: I tensorflow/compiler/xla/service/service.cc:176]   Stream

...

INFO:tensorflow:global step 7357: loss = 0.6657 (0.184 sec/step)
I1127 16:43:51.622658 140454575892288 learning.py:507] global step 7357: loss = 0.6657 (0.184 sec/step)
INFO:tensorflow:global step 7358: loss = 1.2353 (0.156 sec/step)
I1127 16:43:51.780011 140454575892288 learning.py:507] global step 7358: loss = 1.2353 (0.156 sec/step)
INFO:tensorflow:global step 7359: loss = 0.7213 (0.177 sec/step)
I1127 16:43:51.958453 140454575892288 learning.py:507] global step 7359: loss = 0.7213 (0.177 sec/step)
INFO:tensorflow:global step 7360: loss = 1.3543 (0.185 sec/step)
I1127 16:43:52.145531 140454575892288 learning.py:507] global step 7360: loss = 1.3543 (0.185 sec/step)
INFO:tensorflow:global step 7361: loss = 1.2628 (0.186 sec/step)
I1127 16:43:52.333323 140454575892288 learning.py:507] global step 7361: loss = 1.2628 (0.186 sec/step)
INFO:tensorflow:global step 7362: loss = 0.3914 (0.168 sec/step)
I1127 16:43:52.503085 140454575892288 learning.py:507] global step 7362: loss = 0.3914 (0.

INFO:tensorflow:global step 7406: loss = 0.3015 (0.184 sec/step)
I1127 16:44:00.304643 140454575892288 learning.py:507] global step 7406: loss = 0.3015 (0.184 sec/step)
INFO:tensorflow:global step 7407: loss = 0.3990 (0.180 sec/step)
I1127 16:44:00.486393 140454575892288 learning.py:507] global step 7407: loss = 0.3990 (0.180 sec/step)
INFO:tensorflow:global step 7408: loss = 1.1166 (0.168 sec/step)
I1127 16:44:00.655679 140454575892288 learning.py:507] global step 7408: loss = 1.1166 (0.168 sec/step)
INFO:tensorflow:global step 7409: loss = 1.3930 (0.165 sec/step)
I1127 16:44:00.822415 140454575892288 learning.py:507] global step 7409: loss = 1.3930 (0.165 sec/step)
INFO:tensorflow:global step 7410: loss = 1.5571 (0.149 sec/step)
I1127 16:44:00.972444 140454575892288 learning.py:507] global step 7410: loss = 1.5571 (0.149 sec/step)
INFO:tensorflow:global step 7411: loss = 0.9090 (0.148 sec/step)
I1127 16:44:01.122053 140454575892288 learning.py:507] global step 7411: loss = 0.9090 (0.

INFO:tensorflow:global step 7455: loss = 0.5383 (0.187 sec/step)
I1127 16:44:08.614216 140454575892288 learning.py:507] global step 7455: loss = 0.5383 (0.187 sec/step)
INFO:tensorflow:global step 7456: loss = 0.7133 (0.171 sec/step)
I1127 16:44:08.786338 140454575892288 learning.py:507] global step 7456: loss = 0.7133 (0.171 sec/step)
INFO:tensorflow:global step 7457: loss = 1.6560 (0.158 sec/step)
I1127 16:44:08.945539 140454575892288 learning.py:507] global step 7457: loss = 1.6560 (0.158 sec/step)
INFO:tensorflow:global step 7458: loss = 1.3772 (0.162 sec/step)
I1127 16:44:09.108995 140454575892288 learning.py:507] global step 7458: loss = 1.3772 (0.162 sec/step)
INFO:tensorflow:global step 7459: loss = 1.0235 (0.194 sec/step)
I1127 16:44:09.304842 140454575892288 learning.py:507] global step 7459: loss = 1.0235 (0.194 sec/step)
INFO:tensorflow:global step 7460: loss = 1.4177 (0.267 sec/step)
I1127 16:44:09.573230 140454575892288 learning.py:507] global step 7460: loss = 1.4177 (0.

INFO:tensorflow:global step 7503: loss = 1.1352 (0.190 sec/step)
I1127 16:44:16.933423 140454575892288 learning.py:507] global step 7503: loss = 1.1352 (0.190 sec/step)
INFO:tensorflow:global step 7504: loss = 1.7352 (0.157 sec/step)
I1127 16:44:17.091992 140454575892288 learning.py:507] global step 7504: loss = 1.7352 (0.157 sec/step)
INFO:tensorflow:global step 7505: loss = 0.9924 (0.237 sec/step)
I1127 16:44:17.330059 140454575892288 learning.py:507] global step 7505: loss = 0.9924 (0.237 sec/step)
INFO:tensorflow:global step 7506: loss = 0.3339 (0.175 sec/step)
I1127 16:44:17.506711 140454575892288 learning.py:507] global step 7506: loss = 0.3339 (0.175 sec/step)
INFO:tensorflow:global step 7507: loss = 0.5988 (0.173 sec/step)
I1127 16:44:17.681423 140454575892288 learning.py:507] global step 7507: loss = 0.5988 (0.173 sec/step)
INFO:tensorflow:global step 7508: loss = 1.0263 (0.154 sec/step)
I1127 16:44:17.837283 140454575892288 learning.py:507] global step 7508: loss = 1.0263 (0.

INFO:tensorflow:global step 7552: loss = 0.8348 (0.200 sec/step)
I1127 16:44:25.140743 140454575892288 learning.py:507] global step 7552: loss = 0.8348 (0.200 sec/step)
INFO:tensorflow:global step 7553: loss = 1.0207 (0.204 sec/step)
I1127 16:44:25.346469 140454575892288 learning.py:507] global step 7553: loss = 1.0207 (0.204 sec/step)
INFO:tensorflow:global step 7554: loss = 1.2656 (0.163 sec/step)
I1127 16:44:25.510886 140454575892288 learning.py:507] global step 7554: loss = 1.2656 (0.163 sec/step)


In command line, inside container, run `tensorboard --logdir=/happy-walrus/models/version_2/training`

### Export model

In [12]:
!ls -l {TRAIN_PATH}

total 130376
-rw-r--r-- 1 root root      3726 Nov 25 21:14 autoaugm_inceptionv2.config
-rw-r--r-- 1 root root       203 Nov 26 13:57 checkpoint
-rw-r--r-- 1 root root  12079326 Nov 26 13:57 events.out.tfevents.1574776637.5df86dfc3c25
-rw-r--r-- 1 root root  11785420 Nov 26 13:57 graph.pbtxt
-rw-r--r-- 1 root root 103337376 Nov 26 13:57 model.ckpt-0.data-00000-of-00001
-rw-r--r-- 1 root root     25565 Nov 26 13:57 model.ckpt-0.index
-rw-r--r-- 1 root root   6249782 Nov 26 13:57 model.ckpt-0.meta
-rw-r--r-- 1 root root      3726 Nov 26 13:56 pipeline.config


In [16]:
ckpt_step=0
!python {OD_PATH}export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path {CONFIG_PATH} \
    --trained_checkpoint_prefix {TRAIN_PATH}model.ckpt-{ckpt_step} \
    --output_directory {GRAPH_PATH}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1126 14:02:04.583538 140266282092352 module_wrapper.py:139] From /models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1126 14:02:04.591241 140266282092352 module_wrapper.py:139] From /models/research/object_detection/exporter.py:389: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1126 14:02:04.591506 140266282092352 module_wrapper.py:139] From /models/research/object_detection/exporter.py:121: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W1126 14:02:04.64


W1126 14:02:09.914073 140266282092352 module_wrapper.py:139] From /models/research/object_detection/exporter.py:268: The name tf.add_to_collection is deprecated. Please use tf.compat.v1.add_to_collection instead.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W1126 14:02:09.914438 140266282092352 deprecation.py:323] From /models/research/object_detection/exporter.py:370: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step

W1126 14:02:09.918906 140266282092352 module_wrapper.py:139] From /models/research/object_detection/exporter.py:402: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README

250 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*,.*Initializer.*,.*Regularizer.*,.*BiasAdd.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
Incomplete shape.

Doc:
scope: The nodes in the model graph are organiz


W1126 14:02:12.294680 140266282092352 module_wrapper.py:139] From /models/research/object_detection/exporter.py:419: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.


W1126 14:02:13.548985 140266282092352 module_wrapper.py:139] From /models/research/object_detection/exporter.py:332: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-11-26 14:02:13.549601: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2019-11-26 14:02:13.549646: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2019-11-26 14:02:13.549684: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5df86dfc3c25): /proc/driver/nvidia/version does not exist
2019-11-26 14:02:13.550022: I tensorflow/core/platform


W1126 14:02:20.461877 140266282092352 module_wrapper.py:139] From /models/research/object_detection/utils/config_util.py:188: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

INFO:tensorflow:Writing pipeline config file to /happy-walrus/models/version_3/inference_graph/autoaugm_inceptionv2/pipeline.config
I1126 14:02:20.462159 140266282092352 config_util.py:190] Writing pipeline config file to /happy-walrus/models/version_3/inference_graph/autoaugm_inceptionv2/pipeline.config


In [17]:
!ls -lh {GRAPH_PATH}

total 102M
-rw-r--r-- 1 root root   77 Nov 26 14:02 checkpoint
-rw-r--r-- 1 root root  51M Nov 26 14:02 frozen_inference_graph.pb
-rw-r--r-- 1 root root  50M Nov 26 14:02 model.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root  16K Nov 26 14:02 model.ckpt.index
-rw-r--r-- 1 root root 2.0M Nov 26 14:02 model.ckpt.meta
-rw-r--r-- 1 root root 3.2K Nov 26 14:02 pipeline.config
drwxr-xr-x 3 root root 4.0K Nov 26 14:02 saved_model


Download model and store it in Google Drive for use in next section (not necessary if model is already in disk).

In [21]:
zip_name = GRAPH_PATH + MODEL_NAME[:-1] + '.zip'
!zip -qr {zip_name} {GRAPH_PATH}

# 2. Evaluation

### Download model (only if model is not on disk)

In [23]:
# Download model
model_id = "1JVFhXqaayxWOIhuQdzmjIMBAHrAhLcEQ"
model_name = "inference_graph.zip"
gdown.download(url="https://drive.google.com/uc?id=" + model_id,
               output=MAIN_PATH + model_name,
               quiet=False)

# Unzip model
!unzip -q {MAIN_PATH}{model_name} -d {MAIN_PATH}

Downloading...
From: https://drive.google.com/uc?id=1JVFhXqaayxWOIhuQdzmjIMBAHrAhLcEQ
To: /happy-walrus/models/version_2/inference_graph.zip
144MB [00:01, 74.9MB/s] 


### Perform inference

To `Object_detection_image.py`:
1. add before last line `cv2.imwrite('/path/to/save/boxed_image.JPG', image)` and comment last line out.
1. `NUM_CLASSES`
1. `IMAGE_NAME`
1. Replace `from utils` with `from object_detection.utils`
1. Modify so it draws boxes in all images in `images/test`

In [23]:
!mkdir {EVAL_PATH}
!mkdir {EVAL_PATH}bboxes
!cat {MAIN_PATH}Object_detection_image.py

######## Image Object Detection Using Tensorflow-trained Classifier #########
#
# Author: Evan Juras
# Date: 1/15/18
# Description: 
# This program uses a TensorFlow-trained neural network to perform object detection.
# It loads the classifier and uses it to perform object detection on an image.
# It draws boxes, scores, and labels around the objects of interest in the image.

## Some of the code is copied from Google's example at
## https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb

## and some is copied from Dat Tran's example at
## https://github.com/datitran/object_detector_app/blob/master/object_detection_app.py

## but I changed it to make it more understandable to me.

# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys
import re

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from object_detection.utils import 

Inference:
- If `NoneType` related error, reinstall opencv with `apt install opencv-python`.
- Change save path to model specific.

In [33]:
!python {MAIN_PATH}Object_detection_image.py




2019-11-15 15:32:20.262223: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2019-11-15 15:32:20.262274: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2019-11-15 15:32:20.262312: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (34c587f1c563): /proc/driver/nvidia/version does not exist
2019-11-15 15:32:20.262643: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-11-15 15:32:20.271954: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2100060000 Hz
2019-11-15 15:32:20.273008: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7eb06e0 initialized for platform Host (this does not guarantee that 

/happy-walrus/models/version_2/images/test/ADE_train_00010155.jpg
/happy-walrus/models/version_2/images/test/orange_kitchen.EH.055.jpg
/happy-walrus/models/version_2/images/test/South_American_kitchen.EH.006.jpg
/happy-walrus/models/version_2/images/test/black_kitchen.NM.058.jpg
/happy-walrus/models/version_2/images/test/boiling_water.NM.001.jpg
/happy-walrus/models/version_2/images/test/Indian_kitchen.NM.007.jpg
/happy-walrus/models/version_2/images/test/clean_kitchen.NM.056.jpg
/happy-walrus/models/version_2/images/test/old_kitchen.EH.036.jpg
/happy-walrus/models/version_2/images/test/kitchen_apartment.NM.004.jpg
/happy-walrus/models/version_2/images/test/pintrest_messy_kitchen.EH.011.jpg
/happy-walrus/models/version_2/images/test/ADE_train_00010280.jpg
/happy-walrus/models/version_2/images/test/large_kitchen.EH.003.jpg
/happy-walrus/models/version_2/images/test/blue_kitchen.NM.063.jpg
/happy-walrus/models/version_2/images/test/old_kitchen.EH.004.jpg
/happy-walrus/models/version_2/im

/happy-walrus/models/version_2/images/test/kitchen_apartment.NM.070.jpg
/happy-walrus/models/version_2/images/test/purple_kitchen.EH.009.jpg
/happy-walrus/models/version_2/images/test/green_kitchen.NM.012.jpg
/happy-walrus/models/version_2/images/test/red_kitchen.EH.015.jpg
/happy-walrus/models/version_2/images/test/small_kitchen.EH.060.jpg
/happy-walrus/models/version_2/images/test/green_kitchen.NM.029.jpg
/happy-walrus/models/version_2/images/test/ADE_train_00010242.jpg
/happy-walrus/models/version_2/images/test/kitchen_danger.NM.021.jpg
/happy-walrus/models/version_2/images/test/pintrest_cluttered_kitchen.EH.066.jpg
/happy-walrus/models/version_2/images/test/ADE_train_00010281.jpg
/happy-walrus/models/version_2/images/test/ADE_train_00000615.jpg
/happy-walrus/models/version_2/images/test/ADE_train_00010513.jpg
/happy-walrus/models/version_2/images/test/clean_kitchen.NM.062.jpg
/happy-walrus/models/version_2/images/test/linoleum_kitchen.EH.056.jpg
/happy-walrus/models/version_2/image

In [35]:
!zip -rq {MAIN_PATH}evaluation/bboxes.zip {MAIN_PATH}evaluation/bboxes

### Evaluation metrics

Make inferences on test images

In [24]:
!python {OD_PATH}inference/infer_detections.py \
  --input_tfrecord_paths={MAIN_PATH}test.record \
  --output_tfrecord_path={EVAL_PATH}test_detections.tfrecord-00000-of-00001 \
  --inference_graph={GRAPH_PATH}frozen_inference_graph.pb \
  --discard_image_pixels



W1126 14:32:56.438692 140041540327232 module_wrapper.py:139] From /models/research/object_detection/inference/infer_detections.py:57: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1126 14:32:56.438922 140041540327232 module_wrapper.py:139] From /models/research/object_detection/inference/infer_detections.py:57: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1126 14:32:56.439109 140041540327232 module_wrapper.py:139] From /models/research/object_detection/inference/infer_detections.py:65: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-11-26 14:32:56.439965: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2019-11-26 14:32:56.440007: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: 

INFO:tensorflow:Reading graph and building model...
I1126 14:32:56.528734 140041540327232 infer_detections.py:71] Reading graph and building model...

W1126 14:32:56.529096 140041540327232 module_wrapper.py:139] From /models/research/object_detection/inference/detection_inference.py:68: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.


W1126 14:32:56.625202 140041540327232 module_wrapper.py:139] From /models/research/object_detection/inference/detection_inference.py:70: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.


W1126 14:32:57.404529 140041540327232 module_wrapper.py:139] From /models/research/object_detection/inference/detection_inference.py:76: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

INFO:tensorflow:Running inference and writing output to /happy-walrus/models/version_3/evaluation/autoaugm_inceptionv2/test_detections.tfrecord-00000-of-00001
I1126 14:32:57.427958 14

Calculate performance metrics.
- There are two documented bugs, solved implementing [this solution](https://github.com/tensorflow/models/issues/3252#issuecomment-363669586) and [this one](https://github.com/tensorflow/models/issues/5924#issuecomment-455081147).
    - Replace line 162 in `metrics/offline_eval_map_corloc.py` for `input_config = configs['eval_input_configs'][0]`.
    - Replace lines 47 to 49 in `metrics/tf_example_parser.py` with:
        ```python
            if tf_example.features.feature[self.field_name].HasField("bytes_list"):
                result = tf_example.features.feature[self.field_name].bytes_list.value
                result = "".join([x if type(x)=='str' else x.decode('utf-8') for x in result])
            else:
                result = None
            return result
        ```

- Adapt evaluation path.

In [153]:
%%bash
OD_PATH='/models/research/object_detection/'
MAIN_PATH='/happy-walrus/models/version_3/'
IMG_PATH=${MAIN_PATH}images/
MODEL_NAME=autoaugm_inceptionv2/
TRAIN_PATH=${MAIN_PATH}training/${MODEL_NAME}
EVAL_PATH=${MAIN_PATH}evaluation/${MODEL_NAME}
GRAPH_PATH=${MAIN_PATH}inference_graph${MODEL_NAME}
METRICS_PATH=${EVAL_PATH}eval_metrics/
# Choose metrics set from:
#   - coco_detection_metrics (multiple IoUs)
#   - pascal_voc_detection_metrics (per-class)
METRICS_SET='coco_detection_metrics'

# NUM_SHARDS=1  # Set to NUM_GPUS if using the parallel evaluation script
# MAIN_PATH='/happy-walrus/models/version_3/'
# OD_PATH='/models/research/object_detection/'
# EVAL_PATH=${MAIN_PATH}evaluation/eval_metrics/

mkdir -p ${METRICS_PATH}

echo "
label_map_path: '/happy-walrus/models/version_3/training/labelmap.pbtxt'
tf_record_input_reader: { input_path: '${EVAL_PATH}test_detections.tfrecord-00000-of-00001' }
" > ${METRICS_PATH}test_input_config.pbtxt

echo "
metrics_set: '${METRICS_SET}'
" > ${METRICS_PATH}test_eval_config.pbtxt

python ${OD_PATH}metrics/offline_eval_map_corloc.py \
  --eval_dir=${METRICS_PATH} \
  --eval_config_path=${METRICS_PATH}test_eval_config.pbtxt \
  --input_config_path=${METRICS_PATH}test_input_config.pbtxt

creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.35s).
Accumulating evaluation results...
DONE (t=0.86s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.005
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.024
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.024
 

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1126 17:54:34.542032 139952838006592 module_wrapper.py:139] From /models/research/object_detection/utils/config_util.py:236: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1126 17:54:34.544550 139952838006592 module_wrapper.py:139] From /models/research/object_detection/metrics/offline_eval_map_corloc.py:105: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:Processing file: /happy-walrus/models/version_3/evaluation/autoaugm_inceptionv2/test_detections.tfrecord-00000-of-00001
I1126 17:54:34.544687 139952838006592 offline_eval_map_corloc.py

In [152]:
# pascal_voc_detection_metrics
!csvtool readable {EVAL_PATH}eval_metrics/metrics.csv

PascalBoxes_Precision/mAP@0.5IOU                       0.00037501438874659976 
PascalBoxes_PerformanceByCategory/AP@0.5IOU/cabinet    0.00047055844267115683 
PascalBoxes_PerformanceByCategory/AP@0.5IOU/chair      0.0003943217665615142  
PascalBoxes_PerformanceByCategory/AP@0.5IOU/countertop 2.767196118065942e-05  
PascalBoxes_PerformanceByCategory/AP@0.5IOU/dishwasher 0.0                    
PascalBoxes_PerformanceByCategory/AP@0.5IOU/outlet     0.0                    
PascalBoxes_PerformanceByCategory/AP@0.5IOU/oven       0.000889829811023182   
PascalBoxes_PerformanceByCategory/AP@0.5IOU/sofa       0.0                    
PascalBoxes_PerformanceByCategory/AP@0.5IOU/stool      0.0                    
PascalBoxes_PerformanceByCategory/AP@0.5IOU/stove      0.0006382272565852379  
PascalBoxes_PerformanceByCategory/AP@0.5IOU/utensil    0.0013295346494442471  


In [158]:
# coco_detection_metrics
!csvtool readable {EVAL_PATH}eval_metrics/metrics.csv

DetectionBoxes_Precision/mAP          0.00015217268320537238 
DetectionBoxes_Precision/mAP@.50IOU   0.0005234425083178967  
DetectionBoxes_Precision/mAP@.75IOU   3.643657641689191e-06  
DetectionBoxes_Precision/mAP (small)  0.00015217268320537238 
DetectionBoxes_Precision/mAP (medium) -1.0                   
DetectionBoxes_Precision/mAP (large)  -1.0                   
DetectionBoxes_Recall/AR@1            0.0007859667835428641  
DetectionBoxes_Recall/AR@10           0.0053512012245757155  
DetectionBoxes_Recall/AR@100          0.02444306841668646    
DetectionBoxes_Recall/AR@100 (small)  0.02444306841668646    
DetectionBoxes_Recall/AR@100 (medium) -1.0                   
DetectionBoxes_Recall/AR@100 (large)  -1.0                   


# References

- [State of the Art Object Detection - Medium](https://medium.com/@lessw/state-of-the-art-object-detection-use-these-top-3-data-augmentations-and-google-brains-optimal-57ac6d8d1de5)
- [Learning Data Augmentation Strategies for Object Detection](https://arxiv.org/abs/1906.11172v1)